<a href="https://colab.research.google.com/github/Minakshi85/Spam-Filter-for-Quora-Questions/blob/master/P3_Spam_filter_for_Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Project 3: Spam filter for Quora questions
Download data from here : https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

Goal : Build a model for identifying if a question on Quora is spam

Suggested Guidelines :

1. To bring down dimensions of your model you can use glove embedding shared with you ( in the data )

2. Here is how you can use pertained embeddings : https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

3. You'll have to Create and maintain your own train/validation splits for the full data shared with you

On Quora, spam is defined as one or more questions, answers, posts, comments, or messages whose purpose appears to be to direct traffic to external commercial sites while providing little to no value back to the Quora Community.

Answers that refer to external websites to answer the question should provide a summary of how the reference answers the question. If the gist of the answer is not understandable by the writing on Quora, then the question will violate answering policies and guidelines[1]. Answers that drive traffic to external sites for promotional or commercial purposes and do not sufficiently answer the question are considered spam.

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [ ]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2024-03-03 03:05:47--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.80.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.80.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2024-03-03 03:05:47--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc975a70d5b92ca1a8f78c1b7ce6.dl.dropboxusercontent.com/cd/0/inline/COUt7ZnQWOItASKzPVa5NoXYfSXI7dCIsAWPXEbkrD6tG4CKWZFqht6MJdNGUw8m7MS5QWjgr9SD1NIkLOIfkP7yjX1uCeBtKcw3adzcMSNxrCIehWBF6ReOcp89rLWvS98/file# [following]
--2024-03-03 03:05:48--  https://uc975a70d5b92ca1a8f78c1b7ce6.dl.dropboxusercontent.com/cd/0/inline/COUt7ZnQWOItASKzPVa5NoXYfSXI7dCIsAWPXEbkrD6

In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2024-03-03 03:05:57--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2024-03-03 03:05:57--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2024-03-03 03:05:58--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
replace glove.42B.300d.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

NameError: name 'np' is not defined

In [ ]:
question = pd.read_csv(r'/content/train.csv?dl=0')

In [ ]:
print(question.shape)
question.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [ ]:
X = question['question_text']
Y = question['target']
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1044897,), (1044897,), (261225,), (261225,))

In [ ]:
y_test.value_counts()

0    245130
1     16095
Name: target, dtype: int64

In [ ]:
#!pip install imbalanced-learn
import imblearn
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
#class_weight = class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
class_weight = compute_class_weight(class_weight='balanced',classes=np.unique(y_train),y=y_train)
#class_weight = imblearn.combine.compute_class_weight('balanced', np.unique(y_train), y_train)

#print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5330117263936698, 1: 8.073066522444565}

In [ ]:
x_train

267796     What would be the blast radius if the Yellowst...
1214248    Why does Chegg offer lower payment for Chemist...
552539     I have tried calling you many times but your p...
1365       Why is music the answer and how do certain not...
256864                 Does gastric problem cause knee pain?
                                 ...                        
871387      Why did people actually follow Pol Pot's orders?
59076      Canada appears heavenly to me—but what should ...
845547     Will Trump's strong policy against illegal imm...
1105159    I'm a MCA (Master in Computer Applications) 1s...
1117762        What are the best probiotic brands on Amazon?
Name: question_text, Length: 1044897, dtype: object

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [ ]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [ ]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [ ]:
x_train

267796     What would be the blast radius if the Yellowst...
1214248    Why does Chegg offer lower payment for Chemist...
552539     I have tried calling you many times but your p...
1365       Why is music the answer and how do certain not...
256864                 Does gastric problem cause knee pain?
                                 ...                        
871387      Why did people actually follow Pol Pot's orders?
59076      Canada appears heavenly to me—but what should ...
845547     Will Trump's strong policy against illegal imm...
1105159    I'm a MCA (Master in Computer Applications) 1s...
1117762        What are the best probiotic brands on Amazon?
Name: question_text, Length: 1044897, dtype: object

In [ ]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)
print(len(seq_test), len(seq_train))
seq_train[0]

261225 1044897


[2, 35, 25, 1, 8575, 3773, 20, 1, 15259, 1173, 7523, 20198]

In [ ]:
vocab_size=len(tk.word_index)
vocab_size

196023

In [ ]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 45), (261225, 45))

In [ ]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index
# that we downloaded

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

# if there are specific words which are not present in pretrained embedding
# their weights will remain 0. if there are too many such words
# then you should probably not use pretrained embeddings


In [ ]:
# Now while creating the model for text data , we'll keep embedding layer weights
# non-trainable as we are supplying those values from our end

inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_input (InputLayer)     [(None, 45)]              0         
                                                                 
 embedding (Embedding)       (None, 45, 300)           58807200  
                                                                 
 lstm (LSTM)                 (None, 512)               1665024   
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_1 (Dropout)         (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 128)               32896 

In [ ]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1,
                             save_best_only=True)

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              batch_size=10000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

NameError: name 'model' is not defined

In [ ]:
best_model = load_model('/content/weights-01-0.2338.h5')

In [ ]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58783200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [ ]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9528024281838164

In [ ]:
p

array([[0.39858234],
       [0.11947393],
       [0.6753924 ],
       ...,
       [0.01505166],
       [0.01708287],
       [0.29535776]], dtype=float32)